## LightGBM

In [17]:
import pandas as pd
import numpy as np

import lightgbm as lgbm

In [20]:
df4 = pd.read_csv('/workspace/kaggle_data/train.csv')
df4.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [12]:
df3 = pd.read_csv('/workspace/kaggle_data/seed.csv')
df3.head()

,text_id,fold
0,0016926B079C,2
1,0022683E9EA5,0
2,00299B378633,1
3,003885A45F42,3
4,0049B1DF5CCC,3


In [8]:
df1 = pd.read_csv('/workspace/kaggle_data/conventions_eda_data.csv')
df1.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,word_count,miss_count,bos_count,week_months,small_count,miss_rate,small_rate,log_miss_rate,log_small_rate,sample
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,262,0,15,0,15,0.000000,0.057252,0.000000,0.055673,0.055673
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,533,1,4,0,4,0.001876,0.007505,0.001874,0.007477,0.010288
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,320,4,2,0,2,0.012500,0.006250,0.012423,0.006231,0.024864
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,727,1,2,0,2,0.001376,0.002751,0.001375,0.002747,0.004809
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,234,7,0,0,0,0.029915,0.000000,0.029476,0.000000,0.044214


In [6]:
df2 = pd.read_csv('/workspace/kaggle_data/DeBERTa-best/fold0.csv')
df2.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,3.185547,3.343750,3.246094,3.458984,3.369141,3.041016
1,2.638672,2.666016,2.943359,2.599609,2.441406,2.775391
2,2.939453,3.039062,3.048828,3.005859,3.058594,3.039062
3,3.912109,3.730469,3.878906,3.707031,3.699219,3.781250
4,2.572266,2.509766,2.777344,2.564453,2.486328,2.435547


In [21]:
df2['word_count'] = df1['word_count']
df2['miss_count'] = df1['miss_count']
df2['bos_count'] = df1['bos_count']
df2['week_months'] = df1['week_months']
df2['small_count'] = df1['small_count']
df2['miss_rate'] = df1['miss_rate']
df2['small_rate'] = df1['small_rate']
df2['log_miss_rate'] = df1['log_miss_rate']
df2['log_small_rate'] = df1['log_small_rate']
df2['sample'] = df1['sample']
df2['fold'] = df3['fold']
df2['target'] = df4['conventions']
df2.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,word_count,miss_count,bos_count,week_months,small_count,miss_rate,small_rate,log_miss_rate,log_small_rate,sample,fold,target
0,3.185547,3.343750,3.246094,3.458984,3.369141,3.041016,262,0,15,0,15,0.000000,0.057252,0.000000,0.055673,0.055673,2,3.0
1,2.638672,2.666016,2.943359,2.599609,2.441406,2.775391,533,1,4,0,4,0.001876,0.007505,0.001874,0.007477,0.010288,0,2.5
2,2.939453,3.039062,3.048828,3.005859,3.058594,3.039062,320,4,2,0,2,0.012500,0.006250,0.012423,0.006231,0.024864,1,2.5
3,3.912109,3.730469,3.878906,3.707031,3.699219,3.781250,727,1,2,0,2,0.001376,0.002751,0.001375,0.002747,0.004809,3,5.0
4,2.572266,2.509766,2.777344,2.564453,2.486328,2.435547,234,7,0,0,0,0.029915,0.000000,0.029476,0.000000,0.044214,3,2.5


In [28]:
fold = 3

df_train = df2[df2.fold != fold]
df_valid = df2[df2.fold == fold]

# 学習データ
x_train = df_train.drop('target',axis = 1)
y_train = df_train['target']

# 評価用データ
x_valid = df_valid.drop('target',axis = 1)
y_valid = df_valid['target']

In [29]:
categorical_features = []

#LightGBM用の訓練データセットを作成
lgb_train = lgbm.Dataset(x_train,y_train,categorical_feature=categorical_features,free_raw_data=False)

#LightGBM用の評価データセットを作成
lgb_eval = lgbm.Dataset(x_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features,free_raw_data=False)


#ハイパーパラメータを設定
params = {
    'boosting_type': 'gbdt',

    #'objective': 'binary', 二値分類
    'objective': 'regression', #回帰
    
    'metric':'rmse',
    #'metric': {'l2', 'l1'}, 

    #'num_leaves': 10,
    #'learning_rate': 0.05,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'vervose': 0
    
}

#LightGBMの学習
lgbm_model = lgbm.train(params,
                        lgb_train,
                        valid_sets=[lgb_train,lgb_eval],
                        verbose_eval=10,
                        num_boost_round=100,
                        early_stopping_rounds= 20
                       )


#最もスコアの良いモデルの保存
optimum_boost_rounds = lgbm_model.best_iteration

#LightGBMの予測
y_pred = lgbm_model.predict(x_valid,num_interation=lgbm_model.best_iteration)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3147
[LightGBM] [Info] Number of data points in the train set: 2933, number of used features: 17
[LightGBM] [Info] Start training from score 3.088135
Training until validation scores don't improve for 20 rounds
[10]	training's rmse: 0.443351	valid_1's rmse: 0.457951
[20]	training's rmse: 0.391318	valid_1's rmse: 0.419275
[30]	training's rmse: 0.368455	valid_1's rmse: 0.413935
[40]	training's rmse: 0.352485	valid_1's rmse: 0.414763
[50]	training's rmse: 0.339335	valid_1's rmse: 0.414923
Early stopping, best iteration is:
[32]	training's rmse: 0.364945	valid_1's rmse: 0.413216
[LightGBM] [Warning] Unknown parameter: num_interation


/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [ ]:
0.44905
0.42415
0.430138
0.413216